In [6]:
!pip install pandas -q
!pip install rouge -q

In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('gptj_test_generated_with_reasoning.csv')

df.head()

,Context,Question,Legal Reasoning,Answer,Prompt,Generated Text
0,and Statistical Manual of Mental Disorders 446...,Can the defendants force Dr. Nadel to provide ...,The issue at hand pertains to the compulsion o...,"No, according to the Graham v. Gielchinsky cas...",Context:\nand Statistical Manual of Mental Dis...,Context:\nand Statistical Manual of Mental Dis...
1,"one of his trial counsel as a witness, so the ...",Did the Appellee's trial counsel have knowledg...,The question at hand pertains to the knowledge...,It is uncertain whether the Appellee's trial c...,Context:\none of his trial counsel as a witnes...,Context:\none of his trial counsel as a witnes...
2,Some of Cruikshank’s objections on appeal appe...,Can Cruikshank's objections on the trial court...,The issue at hand pertains to the admissibilit...,"No, Cruikshank's objections on the trial court...",Context:\nSome of Cruikshank’s objections on a...,Context:\nSome of Cruikshank’s objections on a...
3,noting that the additional materials provided ...,Can the motion to strike filed by PFM be consi...,The question pertains to the categorization of...,"Yes, the motion to strike filed by PFM can be ...",Context:\nnoting that the additional materials...,Context:\nnoting that the additional materials...
4,The State is within the powers reserved to it ...,Can the State refuse to enter into agreements ...,The question revolves around the State's power...,"Yes, the State has the power to refuse to ente...",Context:\nThe State is within the powers reser...,Context:\nThe State is within the powers reser...


In [16]:
import re

# Function to extract answer using regular expression.
def extract_answer(text):
    match = re.search(r'Answer:\n(.*?)(?:\n|$)', text)
    if match:
        return match.group(1)
    return None

# Apply the function to the 'Generated Text' column.
df['Generated Answer'] = df['Generated Text'].apply(extract_answer)

df.head()

,Context,Question,Legal Reasoning,Answer,Prompt,Generated Text,Generated Answer
0,and Statistical Manual of Mental Disorders 446...,Can the defendants force Dr. Nadel to provide ...,The issue at hand pertains to the compulsion o...,"No, according to the Graham v. Gielchinsky cas...",Context:\nand Statistical Manual of Mental Dis...,Context:\nand Statistical Manual of Mental Dis...,"No, as Dr. Nadel's testimony would have been p..."
1,"one of his trial counsel as a witness, so the ...",Did the Appellee's trial counsel have knowledg...,The question at hand pertains to the knowledge...,It is uncertain whether the Appellee's trial c...,Context:\none of his trial counsel as a witnes...,Context:\none of his trial counsel as a witnes...,"No, the record is silent regarding the potenti..."
2,Some of Cruikshank’s objections on appeal appe...,Can Cruikshank's objections on the trial court...,The issue at hand pertains to the admissibilit...,"No, Cruikshank's objections on the trial court...",Context:\nSome of Cruikshank’s objections on a...,Context:\nSome of Cruikshank’s objections on a...,"No, Cruikshank's objections on the trial court..."
3,noting that the additional materials provided ...,Can the motion to strike filed by PFM be consi...,The question pertains to the categorization of...,"Yes, the motion to strike filed by PFM can be ...",Context:\nnoting that the additional materials...,Context:\nnoting that the additional materials...,"No, the motion to strike can not be considered..."
4,The State is within the powers reserved to it ...,Can the State refuse to enter into agreements ...,The question revolves around the State's power...,"Yes, the State has the power to refuse to ente...",Context:\nThe State is within the powers reser...,Context:\nThe State is within the powers reser...,"Based on case law and federal court rulings, i..."


In [17]:
#drop na

df.dropna(inplace=True)

In [18]:
from rouge import Rouge

# Initialize the Rouge object
rouge = Rouge()

# Function to compute ROUGE scores between 'Answer' and 'Actual Answer' columns
def compute_rouge_scores(row):
    scores = rouge.get_scores(row['Generated Answer'], row['Answer'])
    return scores[0]

# Apply the function to each row of the dataframe
df['ROUGE Scores'] = df.apply(compute_rouge_scores, axis=1)

# Display the dataframe with the ROUGE scores
print(df['ROUGE Scores'])

0       {'rouge-1': {'r': 0.3448275862068966, 'p': 0.4...
1       {'rouge-1': {'r': 0.43333333333333335, 'p': 0....
2       {'rouge-1': {'r': 0.4444444444444444, 'p': 0.8...
3       {'rouge-1': {'r': 0.6666666666666666, 'p': 0.3...
4       {'rouge-1': {'r': 0.35, 'p': 0.297872340425531...
                              ...                        
1492    {'rouge-1': {'r': 0.5897435897435898, 'p': 0.5...
1493    {'rouge-1': {'r': 0.5098039215686274, 'p': 0.5...
1494    {'rouge-1': {'r': 0.46153846153846156, 'p': 0....
1495    {'rouge-1': {'r': 0.5319148936170213, 'p': 0.5...
1496    {'rouge-1': {'r': 0.26666666666666666, 'p': 0....
Name: ROUGE Scores, Length: 1431, dtype: object


In [19]:
# Calculate average ROUGE scores
total_rows = len(df)
total_rouge_1_f1, total_rouge_2_f1, total_rouge_l_f1 = 0, 0, 0

for row in df['ROUGE Scores']:
    total_rouge_1_f1 += row['rouge-1']['f']
    total_rouge_2_f1 += row['rouge-2']['f']
    total_rouge_l_f1 += row['rouge-l']['f']

average_rouge_1_f1 = total_rouge_1_f1 / total_rows
average_rouge_2_f1 = total_rouge_2_f1 / total_rows
average_rouge_l_f1 = total_rouge_l_f1 / total_rows

print("Average ROUGE-1 F1 Score:", average_rouge_1_f1)
print("Average ROUGE-2 F1 Score:", average_rouge_2_f1)
print("Average ROUGE-L F1 Score:", average_rouge_l_f1)

Average ROUGE-1 F1 Score: 0.48751423067966293
Average ROUGE-2 F1 Score: 0.3079589279053287
Average ROUGE-L F1 Score: 0.4460853282167372


In [20]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
# Load the pre-trained SentenceTransformer model
model = SentenceTransformer('Stern5497/sbert-legal-xlm-roberta-base')

# Calculate the semantic similarity between 'Answer' and 'Actual Answer' columns
similarity_scores = []

for i, row in df.iterrows():
    answer_embedding = model.encode([row['Generated Answer']])
    actual_answer_embedding = model.encode([row['Answer']])
    similarity = cosine_similarity(answer_embedding, actual_answer_embedding)
    similarity_scores.append(similarity[0][0])

# Add the similarity scores to the dataframe
df['Semantic Similarity'] = similarity_scores

2023-08-19 12:48:37.270011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 12:48:38.031237: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [22]:
df['Semantic Similarity'].mean()

0.9031818

In [24]:
import re

# Function to extract answer using regular expression.
def extract_reasoning(text):
    match = re.search(r'Reasoning:\n(.*?)(?:\n|$)', text)
    if match:
        return match.group(1)
    return None

# Apply the function to the 'Generated Text' column.
df['Generated Reasoning'] = df['Generated Text'].apply(extract_reasoning)

df.head()

,Context,Question,Legal Reasoning,Answer,Prompt,Generated Text,Generated Answer,ROUGE Scores,Semantic Similarity,Generated Reasoning
0,and Statistical Manual of Mental Disorders 446...,Can the defendants force Dr. Nadel to provide ...,The issue at hand pertains to the compulsion o...,"No, according to the Graham v. Gielchinsky cas...",Context:\nand Statistical Manual of Mental Dis...,Context:\nand Statistical Manual of Mental Dis...,"No, as Dr. Nadel's testimony would have been p...","{'rouge-1': {'r': 0.3448275862068966, 'p': 0.4...",0.841340,Testimony from an expert witness is considered...
1,"one of his trial counsel as a witness, so the ...",Did the Appellee's trial counsel have knowledg...,The question at hand pertains to the knowledge...,It is uncertain whether the Appellee's trial c...,Context:\none of his trial counsel as a witnes...,Context:\none of his trial counsel as a witnes...,"No, the record is silent regarding the potenti...","{'rouge-1': {'r': 0.43333333333333335, 'p': 0....",0.957416,This question focuses on the Appellee's trial ...
2,Some of Cruikshank’s objections on appeal appe...,Can Cruikshank's objections on the trial court...,The issue at hand pertains to the admissibilit...,"No, Cruikshank's objections on the trial court...",Context:\nSome of Cruikshank’s objections on a...,Context:\nSome of Cruikshank’s objections on a...,"No, Cruikshank's objections on the trial court...","{'rouge-1': {'r': 0.4444444444444444, 'p': 0.8...",0.941405,The purpose of raising objections on appeal is...
3,noting that the additional materials provided ...,Can the motion to strike filed by PFM be consi...,The question pertains to the categorization of...,"Yes, the motion to strike filed by PFM can be ...",Context:\nnoting that the additional materials...,Context:\nnoting that the additional materials...,"No, the motion to strike can not be considered...","{'rouge-1': {'r': 0.6666666666666666, 'p': 0.3...",0.851548,The question pertains to whether the motion to...
4,The State is within the powers reserved to it ...,Can the State refuse to enter into agreements ...,The question revolves around the State's power...,"Yes, the State has the power to refuse to ente...",Context:\nThe State is within the powers reser...,Context:\nThe State is within the powers reser...,"Based on case law and federal court rulings, i...","{'rouge-1': {'r': 0.35, 'p': 0.297872340425531...",0.635518,The question pertains to the issue of whether ...


In [25]:
from rouge import Rouge

# Initialize the Rouge object
rouge = Rouge()

# Function to compute ROUGE scores between 'Answer' and 'Actual Answer' columns
def compute_reasoning_rouge_scores(row):
    scores = rouge.get_scores(row['Generated Reasoning'], row['Legal Reasoning'])
    return scores[0]

# Apply the function to each row of the dataframe
df['Reasoning ROUGE Scores'] = df.apply(compute_reasoning_rouge_scores, axis=1)

# Display the dataframe with the ROUGE scores
print(df['Reasoning ROUGE Scores'])

0       {'rouge-1': {'r': 0.37362637362637363, 'p': 0....
1       {'rouge-1': {'r': 0.4431818181818182, 'p': 0.5...
2       {'rouge-1': {'r': 0.20652173913043478, 'p': 0....
3       {'rouge-1': {'r': 0.6233766233766234, 'p': 0.4...
4       {'rouge-1': {'r': 0.29591836734693877, 'p': 0....
                              ...                        
1492    {'rouge-1': {'r': 0.4050632911392405, 'p': 0.5...
1493    {'rouge-1': {'r': 0.4888888888888889, 'p': 0.4...
1494    {'rouge-1': {'r': 0.5303030303030303, 'p': 0.6...
1495    {'rouge-1': {'r': 0.42045454545454547, 'p': 0....
1496    {'rouge-1': {'r': 0.4835164835164835, 'p': 0.4...
Name: Reasoning ROUGE Scores, Length: 1431, dtype: object


In [27]:
total_rows = len(df)
total_rouge_1_f1 = 0
total_rouge_2_f1 = 0
total_rouge_l_f1 = 0
total_rouge_1_precision = 0
total_rouge_2_precision = 0
total_rouge_l_precision = 0
total_rouge_1_recall = 0
total_rouge_2_recall = 0
total_rouge_l_recall = 0

for row in df['ROUGE Scores']:
    total_rouge_1_f1 += row['rouge-1']['f']
    total_rouge_2_f1 += row['rouge-2']['f']
    total_rouge_l_f1 += row['rouge-l']['f']
    total_rouge_1_precision += row['rouge-1']['p']
    total_rouge_2_precision += row['rouge-2']['p']
    total_rouge_l_precision += row['rouge-l']['p']
    total_rouge_1_recall += row['rouge-1']['r']
    total_rouge_2_recall += row['rouge-2']['r']
    total_rouge_l_recall += row['rouge-l']['r']

average_rouge_1_f1 = total_rouge_1_f1 / total_rows
average_rouge_2_f1 = total_rouge_2_f1 / total_rows
average_rouge_l_f1 = total_rouge_l_f1 / total_rows
average_rouge_1_precision = total_rouge_1_precision / total_rows
average_rouge_2_precision = total_rouge_2_precision / total_rows
average_rouge_l_precision = total_rouge_l_precision / total_rows
average_rouge_1_recall = total_rouge_1_recall / total_rows
average_rouge_2_recall = total_rouge_2_recall / total_rows
average_rouge_l_recall = total_rouge_l_recall / total_rows

print("Average ROUGE-1 F1 Score:", average_rouge_1_f1)
print("Average ROUGE-2 F1 Score:", average_rouge_2_f1)
print("Average ROUGE-L F1 Score:", average_rouge_l_f1)
print("Average ROUGE-1 Precision:", average_rouge_1_precision)
print("Average ROUGE-2 Precision:", average_rouge_2_precision)
print("Average ROUGE-L Precision:", average_rouge_l_precision)
print("Average ROUGE-1 Recall:", average_rouge_1_recall)
print("Average ROUGE-2 Recall:", average_rouge_2_recall)
print("Average ROUGE-L Recall:", average_rouge_l_recall)

Average ROUGE-1 F1 Score: 0.48751423067966293
Average ROUGE-2 F1 Score: 0.3079589279053287
Average ROUGE-L F1 Score: 0.4460853282167372
Average ROUGE-1 Precision: 0.5169525186815209
Average ROUGE-2 Precision: 0.33129341283685604
Average ROUGE-L Precision: 0.4734314449052739
Average ROUGE-1 Recall: 0.5095914468356101
Average ROUGE-2 Recall: 0.3319517731626096
Average ROUGE-L Recall: 0.46622241564332095
